#### Importing required Libraries

In [1]:
import transformers
import torch
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score,accuracy_score
print(torch.cuda.is_available())  
print(torch.cuda.device_count())  
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 3050 4GB Laptop GPU


#### Loading the dataset, model and tokenizer

In [2]:

dataset = load_dataset("imdb")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2) # Binary Classifier model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Preprocessing

In [3]:
def tokenize(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)
tokenized_dataset = dataset.map(tokenize,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

#### Fine-tuning BERT model

In [4]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    fp16=True,
    no_cuda=False
)
trainer = Trainer(model=model,args=training_args,train_dataset=tokenized_dataset["train"])
trainer.train()

Step,Training Loss
500,0.366900
1000,0.304300
1500,0.277300
2000,0.285200
2500,0.263000
3000,0.258700
3500,0.192200
4000,0.169900
4500,0.153600
5000,0.149500


TrainOutput(global_step=9375, training_loss=0.17403549682617186, metrics={'train_runtime': 6037.4862, 'train_samples_per_second': 12.422, 'train_steps_per_second': 1.553, 'total_flos': 1.9733329152e+16, 'train_loss': 0.17403549682617186, 'epoch': 3.0})

#### Evaluation of the fine tuned model

In [5]:
predictions_output = trainer.predict(tokenized_dataset["test"])

# Extract logits and true labels
logits = predictions_output.predictions
y_true = predictions_output.label_ids

# Convert logits to predicted class (maximum value)
y_pred = logits.argmax(axis=-1)

print("F1 Score:",f1_score(y_true,y_pred))
print("Accuracy:",accuracy_score(y_true,y_pred))

F1 Score: 0.9400439384861194
Accuracy: 0.93996


#### Saved the model

In [6]:
model.config.id2label = {0: "NEGATIVE", 1: "POSITIVE"}
model.config.label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.txt',
 './fine_tuned_model\\added_tokens.json',
 './fine_tuned_model\\tokenizer.json')

#### Loading the model & Using on a sample text

In [7]:
classifier = pipeline("sentiment-analysis",model="./fine_tuned_model",tokenizer="./fine_tuned_model")
result = classifier("I really like the movie. It was a great thriller")

print(result)

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9997004270553589}]


#### Pushing the model to the Hugging Face Model Hub

In [8]:
from huggingface_hub import login
login()
model.push_to_hub("my-model")
tokenizer.push_to_hub("my-model")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

c:\Users\amann\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amann\.cache\huggingface\hub\models--aman-nehra--my-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/aman-nehra/my-model/commit/61680e86911648fb9a1ced34740274b3841e9dc7', commit_message='Upload tokenizer', commit_description='', oid='61680e86911648fb9a1ced34740274b3841e9dc7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aman-nehra/my-model', endpoint='https://huggingface.co', repo_type='model', repo_id='aman-nehra/my-model'), pr_revision=None, pr_num=None)